# TestCommands
Using something like that:
```shell
sudo ping -O -D -t 25 -v 8.8.8.8 -i 0.05 > pingtest-5G-50msDelay-googledns
sudo ping -O -D -t 25 -v 8.8.8.8 -i 1 > pingtest-5G-1sDelay-googledns

# not working yet and unclear
sudo hping3 mealie.horotw.com --destport 80 --numeric --ack --interval u50000 | xargs -I {} bash -c 'echo "[$EPOCHREALTIME] {}"' > pingtest-50msdelay-hping-mealie.yggdrasil-ack
```

In [ ]:
PATH = "~/Nextcloud/NC/pingtests/"
# Set the number of images to analyze
if "ANALYZE_COUNT" in locals():
    print("ANALYZE_COUNT is defined")
else:
    print("ANALYZE_COUNT is not defined falling back to 12")
    ANALYZE_COUNT = 2

if "COUNT_AS_LOSS_IF_OVER" in locals():
    print("COUNT_AS_LOSS_IF_OVER is defined")
else:
    print("COUNT_AS_LOSS_IF_OVER is not defined falling back to using 5x std deviation")
    COUNT_AS_LOSS_IF_OVER = 75

In [ ]:
from tabulate import tabulate
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML
from parsefuncs import parse_ping_file, parse_hping_file
import pathlib

In [ ]:
pPath = pathlib.Path(PATH).expanduser()
print(f"Glob Path: {str(pPath / 'pingtest-*')}")
data_files = glob.glob(str(pPath / "pingtest-*"))
print("Found {} files".format(len(data_files)))

dfs = []
def read_dfs():
    global dfs, data_files
    dfs = []

    # sort the files by creation time and take the last ANALYZE_COUNT files
    data_files.sort(key=os.path.getctime, reverse=True)
    data_files = data_files[:ANALYZE_COUNT]


    # Read the data using a regular expression to extract the relevant fields
    for data_file in data_files:
        if "hping" in data_file:
            data = parse_hping_file(data_file)
            dfs.append(data)
        else:  # normal ping
            data = parse_ping_file(data_file, COUNT_AS_LOSS_IF_OVER)
            dfs.append(data)

read_dfs()

In [ ]:
# create subplots
fig, axs = plt.subplots(len(dfs), 1, sharex=True, sharey=True)
axs = np.atleast_1d(axs)  # Ensure axs is treated as an array-like object
colors =[ "red", "blue", "green", "orange", "yellow", "pink", "purple", "brown", "gray", "cyan", "magenta","lime"]

for i, df in enumerate(dfs):
    color = colors[i]
    axs[i].plot(df["timestamp"], df["time"], label=data_files[i], color=color)
    axs[i].legend()
    axs[i].grid()

plt.show()

In [ ]:
statistics = []

def get_statistics():
    global statistics, data_files, dfs
    statistics = []

    for i, df in enumerate(dfs):
        loss_count = len(df[df.isna().any(axis=1)])
        stats = {
            "DataFrame": data_files[i],
            "Mean": df["time"].mean(),
            "Median": df["time"].median(),
            "Std Dev": df["time"].std(),
            "Min": df["time"].min(),
            "Max": df["time"].max(),
            "q25": df["time"].quantile(0.25),
            # "q50": df["time"].quantile(0.5),  # same as median
            "q75": df["time"].quantile(0.75),
            "q90": df["time"].quantile(0.9),
            "q95": df["time"].quantile(0.95),
            "q99": df["time"].quantile(0.99),
            "Loss Rate in %": (loss_count / len(df)) * 100,
            "|loss|": loss_count,
        }
        statistics.append(stats)
    return statistics

statistics = get_statistics()

In [ ]:
# better than subplots is unsing FigureWidget
fig2 = ""
fig2 = go.FigureWidget(data=[])

fig2.update_layout(
    title="Ping times",
    xaxis_title="Time",
    yaxis_title="Time in ms",
    legend_title="Legend"
)

"Done"

In [ ]:
# clear all traces
fig2.data = []

In [ ]:
def update_figure(change):
    global fig2
    # clear all traces
    fig2.data = []

    # refresh the figure data
    read_dfs()
    get_statistics()

    shapes = []
    annotations = []
    traces = []
    # add the new data to the figure
    for i, df in enumerate(dfs):
        traces.append(
            go.Scatter(
                x=df["timestamp"],
                y=df["time"],
                name=data_files[i],
                mode="lines+markers",
                marker=dict(color=colors[i], size=3),
                showlegend=True,
                line=dict(color=colors[i], width=1),
            )
        )

        for index, row in df.iterrows():  # for each row
            # to check for NaN use np.isnan(row["time"])
            if np.isnan(row["time"]):
                shapes.append(
                    # Line Vertical
                    dict(
                        type="line",
                        x0=row["timestamp"],
                        y0=0,
                        x1=row["timestamp"],
                        y1=60,
                        line=dict(color=colors[i], width=3),
                    )
                )
                annotations.append(
                    dict(
                        x=row["timestamp"],
                        y=60,
                        text="X",
                        showarrow=True,
                        arrowhead=1,
                        ax=0,
                        ay=-30,
                        font=dict(color="black", size=16),
                    )
                )
    fig2.update_layout(shapes=shapes, annotations=annotations)
    fig2.add_traces(traces)




In [ ]:
# Update the layout to make it more interactive
fig2.update_layout(
    title="Scrub Through Time",
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="minute", stepmode="backward"),
                    dict(count=5, label="5m", step="minute", stepmode="backward"),
                    dict(count=30, label="30m", step="minute", stepmode="backward"),
                    dict(count=1, label="1h", step="hour", stepmode="backward"),
                    dict(count=3, label="3h", step="hour", stepmode="backward"),
                    dict(count=6, label="6h", step="hour", stepmode="backward"),
                    dict(count=12, label="12h", step="hour", stepmode="backward"),
                    dict(count=1, label="1d", step="day", stepmode="backward"),
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    ),
)
# clear this cell's output:
display(HTML("<script>Jupyter.notebook.clear_all_output();</script>"))

fig2 # display the figure

In [ ]:
update_figure(None)
print(f"ANALYZE_COUNT: {ANALYZE_COUNT}")
table = tabulate(statistics, headers="keys", tablefmt="html") 
display(HTML(table))

In [ ]:
#   7.6 + 0.18 = 7.78
#   7.6 + 1.44 = 9.04
ANALYZE_COUNT = 2
COUNT_AS_LOSS_IF_OVER = 75